In [1]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece
!pip install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00


In [3]:
#This downloads the model and defines the function required of inference

%%writefile llm.py

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'filipealmeida/Mistral-7B-Instruct-v0.1-sharded'

def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

# Define stop token ids
stop_token_ids = [0]


def inference(user_input):
    text = f"[INST] {user_input} [/INST]"

    # Move model to appropriate device
    device = next(model.parameters()).device

    encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
    encoded = {k: v.to(device) for k, v in encoded.items()}  # Move tensors to the model's device

    generated_ids = model.generate(**encoded, max_length=9999, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    result_text = decoded[0].split(text)[1].strip()

    return result_text


import locale
locale.getpreferredencoding = lambda: "UTF-8"


from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is Saathi.Your job is to help the user with any task and provide emotional support."),
        ("human", "{user_input}"),
    ]
)

def chat(user_prompt):
  messages = chat_template.format_messages(user_input=user_prompt)
  response=inference(messages)
  return response



Writing llm.py


In [4]:
!python3 llm.py # this will download the quantized model

config.json: 100% 665/665 [00:00<00:00, 2.86MB/s]
pytorch_model.bin.index.json: 100% 23.9k/23.9k [00:00<00:00, 76.6MB/s]
pytorch_model-00001-of-00008.bin:   0% 0.00/1.89G [00:00<?, ?B/s]
pytorch_model-00001-of-00008.bin:   1% 21.0M/1.89G [00:00<00:15, 121MB/s]
pytorch_model-00001-of-00008.bin:   3% 52.4M/1.89G [00:00<00:09, 194MB/s]
pytorch_model-00001-of-00008.bin:   4% 73.4M/1.89G [00:00<00:09, 187MB/s]
pytorch_model-00001-of-00008.bin:   5% 94.4M/1.89G [00:00<00:09, 193MB/s]
pytorch_model-00001-of-00008.bin:   7% 126M/1.89G [00:00<00:09, 192MB/s] 
pytorch_model-00001-of-00008.bin:   8% 147M/1.89G [00:00<00:11, 158MB/s]
pytorch_model-00001-of-00008.bin:   9% 168M/1.89G [00:00<00:10, 158MB/s]
pytorch_model-00001-of-00008.bin:  10% 189M/1.89G [00:01<00:10, 158MB/s]
pytorch_model-00001-of-00008.bin:  11% 210M/1.89G [00:01<00:10, 160MB/s]
pytorch_model-00001-of-00008.bin:  12% 231M/1.89G [00:01<00:10, 156MB/s]
pytorch_model-00001-of-00008.bin:  14% 262M/1.89G [00:01<00:10, 162MB/s]
pytor

In [7]:
# This is the streamlit app

%%writefile app.py


import streamlit as st
import random
import time
from llm import chat

st.title("😇 Saathi ChatBot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input():
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})



# Display assistant response in chat message container
with st.chat_message("assistant"):
    message_placeholder = st.empty()
    full_response = ""
    # getting the user response
    user_messages = [msg['content'] for msg in st.session_state.messages if msg['role'] == 'user']
    assistant_response=""
    if(len(user_messages)>0):
        print(user_messages[-1])
        assistant_response=chat(user_messages[-1])[:-4]


    # Simulate stream of response with milliseconds delay
    for chunk in assistant_response.split():
        full_response += chunk + " "
        time.sleep(0.05)
        # Add a blinking cursor to simulate typing
        message_placeholder.markdown(full_response + "▌")
    message_placeholder.markdown(full_response)
# Add assistant response to chat history
st.session_state.messages.append({"role": "assistant", "content": full_response})




Overwriting app.py


In [8]:
!streamlit run app.py & npx localtunnel --port 8501

[##................] - fetchMetadata: sill resolveWithNewModule require-directo


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.197.61.182:8501

npx: installed 22 in 4.034s
your url is: https://poor-ideas-rhyme.loca.lt
Loading checkpoint shards:   0% 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100% 8/8 [01:41<00:00, 12.66s/it]
hello
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2023-12-21 08:16:42.039992: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: A